# AutoML tested on every file individually (4 cores, 30 seconds per file)
AutoML's data preprocessing is applied to these tests. Results from each model's tests are compiled into a single average.

In [1]:
import sys
sys.path.append("../../")
from time import process_time
from os import listdir, chdir, environ

def warn(*args, **kwargs):
  pass
import warnings
warnings.warn = warn

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
from modules.NetworkTraffic import NT2
from sklearn import model_selection, metrics

from autosklearn.experimental.askl2 import AutoSklearn2Classifier

FilesToTest = list()
chdir("../../data")
for file in listdir():
  if file.endswith(".csv"):
    FilesToTest.append(file)

TestSize = [0.4]
ModelsToTest = [AutoSklearn2Classifier(time_left_for_this_task=30, memory_limit=4096, n_jobs=-1)]
OutputResults = dict()
ModelResults = dict()

In [2]:
OutputResults.clear()
ModelResults.clear()

for size in TestSize:
  print(f"\nSearching with test size of {size*100}%...")
  OutputResults[size] = dict()
  ModelResults[size] = dict()

  for file in FilesToTest:
    print(file, end=', ')
    OutputResults[size][file] = dict()
    currentFileData = NT2(file, transform=False, drop=False)

    for model in ModelsToTest:
      #print(f"{file} : {str(model)}...")
      results = model_selection.cross_val_score(model, currentFileData.data, currentFileData.target, cv=5)
      OutputResults[size][file].update({str(model): {"Accuracy":np.mean(results), "Std Dev":np.std(results)}})


Searching with test size of 40.0%...
b5000d100.csv, b5000d30.csv, b100d10.csv, b1000d10.csv, b1000d100.csv, b100d100.csv, b5000d10.csv, b1000d30.csv, b100d30.csv, [WARNING] [2022-04-26 16:48:07,869:Client-autosklearn.automl_common.common.utils.backend] Directory /tmp/auto-sklearn_tmp_403d0e07-c5bb-11ec-a852-efbf159bdcbc/.auto-sklearn/ensembles does not exist
[WARNING] [2022-04-26 16:48:09,023:Client-AutoML(1):403d0c95-c5bb-11ec-a852-efbf159bdcbc] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:[(1, 3, 0.0)]


In [3]:
import json
with open("EveryFileIndividually_Untuned_AllTestResults.json", "a") as f:
  f.write(json.dumps(OutputResults, indent=2))

In [4]:
with open("EveryFileIndividual_Untuned_ModelResults.csv", "w") as f3:
  f3.write("File,Model,Accuracy,Std Dev\n")
  for size in OutputResults:
    for file in OutputResults[size]:
      for model in OutputResults[size][file]:
        f3.write(f"{file},{model},{OutputResults[size][file][model]['Accuracy']},{OutputResults[size][file][model]['Std Dev']}\n")